# **Laboratorio 8**
- Joaquín Campos - 22155
- Sofía García - 22210
- Julio García Salas - 22076

## **Inciso 1**

In [0]:
# INCISO 1 — con el nuevo maestro: /Volumes/workspace/default/lab8/master_unico.csv
# Versión compatible con Serverless (sin cache/persist)

from pyspark.sql import functions as F
import pandas as pd

# -----------------------------------------------------------
# 0) Paths y verificación
# -----------------------------------------------------------
PARENT = "/Volumes/workspace/default/lab8"
PATH   = "/Volumes/workspace/default/lab8/master_unico.csv"

display(dbutils.fs.ls(PARENT))

print("=== master_unico.csv (inicio) ===")
try:
    raw = dbutils.fs.head(PATH, 2048)
    print(raw[:1000])
except Exception as e:
    print("[WARN] No se pudo leer cabecera con dbutils.fs.head:", e)

# -----------------------------------------------------------
# 1) Carga robusta: pandas (utf-8-sig) -> Spark, con fallback a spark.read.csv
# -----------------------------------------------------------
df_all = None
try:
    pdf = pd.read_csv(PATH, dtype=str, encoding="utf-8-sig", engine="python")
    pdf.columns = [c.strip() for c in pdf.columns]
    df_all = spark.createDataFrame(pdf)
    print("[OK] Cargado con pandas -> Spark")
except Exception as e:
    print("[WARN] pandas.read_csv falló, probando spark.read.csv. Error:", e)
    df_all = (
        spark.read.format("csv")
        .option("header", True)
        .option("multiLine", True)
        .option("escape", '"')
        .load(PATH)
        .toDF(*[c.strip() for c in spark.read.csv(PATH, header=True).columns])
    )
    print("[OK] Cargado con spark.read.csv")

# 🚫 NO usar cache()/persist() en Serverless (no soportado)
# Materializamos una vez con count() y seguimos
total_rows = df_all.count()
print(f"\nTotal filas en master_unico: {total_rows:,}")
print(f"Total columnas: {len(df_all.columns)}")
print("Columnas (primeras 15):", df_all.columns[:15])

# Helper is_empty seguro (evita .isEmpty() si no está disponible)
def is_empty(sdf):
    try:
        return sdf.isEmpty()
    except Exception:
        return sdf.limit(1).count() == 0

# -----------------------------------------------------------
# 2) Separación por “tablas”
# -----------------------------------------------------------
if "source_table" in df_all.columns:
    hechos_df    = df_all.filter(F.col("source_table") == "HECHOS")
    vehiculos_df = df_all.filter(F.col("source_table") == "VEHICULOS")
    personas_df  = df_all.filter(F.col("source_table") == "PERSONAS")
else:
    hechos_df = df_all.filter(F.col("tipo_eve_norm").isNotNull())
    vehiculos_df = df_all.filter(
        F.col("tipo_veh_norm").isNotNull() |
        F.col("color_std").isNotNull() |
        F.col("sexo_pil_norm").isNotNull() |
        F.col("edad_pil_num").isNotNull()
    )
    personas_df = df_all.filter(
        F.col("sexo_per_norm").isNotNull() |
        F.col("edad_per_num").isNotNull() |
        F.col("is_fallecido").isNotNull() |
        F.col("is_lesionado").isNotNull()
    )

print("\n=== Conteos por 'tabla' ===")
if "source_table" in df_all.columns:
    df_all.groupBy("source_table").count().orderBy("source_table").show(truncate=False)
else:
    print(f"Hechos: {hechos_df.count():,} | Vehículos: {vehiculos_df.count():,} | Personas: {personas_df.count():,}")

# -----------------------------------------------------------
# 3) Muestras .show()
# -----------------------------------------------------------
print("\n--- Muestra HECHOS ---")
hechos_cols = [c for c in [
    "anio_norm","mes_norm","dia_norm","hora_clean","franja_horaria","dow_num","dow_label",
    "depto_code","depto_name_std","muni_code","zona_ciudad_num","tipo_eve_norm","anio_mes"
] if c in hechos_df.columns]
hechos_df.select(*hechos_cols).show(10, truncate=False)

print("\n--- Muestra VEHICULOS ---")
veh_cols = [c for c in [
    "anio_norm","mes_norm","hora_clean","depto_code","tipo_eve_norm",
    "tipo_veh_norm","color_std","sexo_pil_norm","edad_pil_num","marca_veh","modelo_veh"
] if c in vehiculos_df.columns]
vehiculos_df.select(*veh_cols).show(10, truncate=False)

print("\n--- Muestra PERSONAS ---")
per_cols = [c for c in [
    "anio_norm","mes_norm","hora_clean","depto_code","tipo_eve_norm",
    "sexo_per_norm","edad_per_num","is_fallecido","is_lesionado"
] if c in personas_df.columns]
personas_df.select(*per_cols).show(10, truncate=False)

# -----------------------------------------------------------
# 4) Describe & Summary de numéricas (sin cache)
# -----------------------------------------------------------
def safe_describe_numeric(sdf, cols, title):
    cols_exist = [c for c in cols if c in sdf.columns]
    if not cols_exist or is_empty(sdf):
        print(f"\n[INFO] Sin filas o columnas para {title}")
        return
    casted = [F.col(c).cast("double").alias(c) for c in cols_exist]
    print(f"\n--- DESCRIBE: {title} ---")
    sdf.select(*casted).describe().show(truncate=False)
    print(f"\n--- SUMMARY: {title} ---")
    sdf.select(*casted).summary().show(truncate=False)

safe_describe_numeric(
    hechos_df,
    ["anio_norm","mes_norm","dia_norm","hora_clean","dow_num","depto_code","muni_code","zona_ciudad_num"],
    "HECHOS (tiempo y ubicación)"
)
safe_describe_numeric(
    vehiculos_df,
    ["anio_norm","depto_code","edad_pil_num"],
    "VEHICULOS (características/conductor)"
)
safe_describe_numeric(
    personas_df,
    ["anio_norm","depto_code","edad_per_num","is_fallecido","is_lesionado"],
    "PERSONAS (atributos/estado)"
)

# -----------------------------------------------------------
# 5) Vistas temporales
# -----------------------------------------------------------
df_all.createOrReplaceTempView("master_unico")
hechos_df.createOrReplaceTempView("hechos")
vehiculos_df.createOrReplaceTempView("vehiculos")
personas_df.createOrReplaceTempView("personas")

print("\n[OK] Vistas creadas: master_unico, hechos, vehiculos, personas (sin cache/persist)")


path,name,size,modificationTime
dbfs:/Volumes/workspace/default/lab8/_UNIFICADO.csv,_UNIFICADO.csv,10395125,1759426297000
dbfs:/Volumes/workspace/default/lab8/master_unico.csv,master_unico.csv,29043186,1759470343000


=== master_unico.csv (inicio) ===
[Truncated to first 2048 bytes]
source_table,anio_norm,mes_norm,dia_norm,hora_clean,franja_horaria,dow_num,dow_label,depto_code,depto_name_std,muni_code,zona_ciudad_num,tipo_eve_norm,anio_mes,color_std,tipo_veh_norm,sexo_pil_norm,edad_pil_num,sexo_per_norm,edad_per_num,is_fallecido,is_lesionado,marca_veh,modelo_veh,color_veh,color_v,tipo_v,modelo_v,source_file
HECHOS,2013,1,1,3,MADRUGADA,2,MARTES,1,GUATEMALA,106,,NAN,2013-01,,,,,,,,,,,,,,,2013.csv
HECHOS,2013,1,1,5,MADRUGADA,2,MARTES,1,GUATEMALA,101,,NAN,2013-01,,,,,,,,,,,,,,,2013.csv
HECHOS,2013,1,1,3,MADRUGADA,2,MARTES,1,GUATEMALA,108,,NAN,2013-01,,,,,,,,,,,,,,,2013.csv
HECHOS,2013,1,1,0,MADRUGADA,2,MARTES,19,ZACAPA,1901,,NAN,2013-01,,,,,,,,,,,,,,,2013.csv
HECHOS,2013,1,1,18,NOCHE,2,MARTES,19,ZACAPA,1901,,NAN,2013-01,,,,,,,,,,,,,,,2013.csv
HECHOS,2013,1,1,0,MADRUGADA,2,MARTES,5,ESCUINTLA,502,,NAN,2013-01,,,,,,,,,,,,,,,2013.csv
HECHOS,2013,1,1,11,MAÑANA,2,MARTES,9,QUETZALTENANGO,901,,NAN,2013-01,,,,,,

# Resumen técnico — `master_unico.csv`

## 1) Estructura general
- **Filas:** 298,164  
- **Columnas:** 29  
- **Tablas unificadas (en `source_table`):**
  - **HECHOS:** 76,759
  - **VEHICULOS:** 111,988
  - **PERSONAS:** 109,417

> El esquema contiene campos normalizados para tiempo (`anio_norm`, `mes_norm`, `dia_norm`, `hora_clean`, `franja_horaria`, `dow_num`, `dow_label`), ubicación (`depto_code`, `depto_name_std`, `muni_code`, `zona_ciudad_num`), tipología (`tipo_eve_norm`, `tipo_veh_norm`, `color_std`) y atributos específicos de vehículo/persona.

---

## 2) Ejemplos de registros (muestras)
- **HECHOS:** filas de 2013 con `MADRUGADA/MAÑANA`, presencia de códigos de departamento y municipio; `tipo_eve_norm` aparece como **`NAN`** en algunas filas tempranas (indica valores faltantes/no mapeados).
- **VEHICULOS:** registros de 2021 con `tipo_eve_norm` ya estandarizado (p.ej., `COLISION`, `DERRAPE`), `tipo_veh_norm` y `color_std` numéricos/categorizados.
- **PERSONAS:** registros de 2021 con `sexo_per_norm`, `edad_per_num` y banderas `is_fallecido`/`is_lesionado`.

---

## 3) Cobertura de campos clave (calidad de datos)
**HECHOS** (base: 76,759 filas)  
- `hora_clean`: 76,732 (≈ **99.96%** cobertura)  
- `dow_num`, `depto_code`, `muni_code`: 60,617 cada uno (≈ **79%** cobertura)  
- `zona_ciudad_num`: 8,153 (≈ **10.6%** cobertura; muy escaso, usar con cuidado)  
- Rango temporal: **2013–2023**

**VEHICULOS** (base: 111,988 filas)  
- `depto_code`: 87,552 (≈ **78.2%**)  
- `edad_pil_num`: 5,382 (≈ **4.8%**; muy baja, sesga análisis por edad del piloto)  
- Rango temporal: **2013–2023**

**PERSONAS** (base: 109,417 filas)  
- `depto_code`: 87,497 (≈ **80.0%**)  
- `edad_per_num`: 84,249 (≈ **77.0%**)  
- `is_fallecido`: media **1.0** y `is_lesionado`: media **0.0** → **la tabla PERSONAS representa fallecidos** (no lesionados).  
- Rango temporal: **2013–2023**

**Implicación:** para preguntas que requieran **lesionados**, se deben derivar desde HECHOS/VEHICULOS (si existe la codificación) o incorporar una fuente adicional; la tabla PERSONAS no aporta lesionados.

---

## 4) Estadística descriptiva (resumen útil)
**HECHOS**  
- `anio_norm` medio ≈ **2018.27**, distribución homogénea 2013–2023.  
- `hora_clean` media ≈ **13.69** (sesgo leve hacia tarde—noche).  
- `dow_num` media ≈ **4.34** (cercano a **jueves**), con ~21% faltantes.

**VEHICULOS**  
- `anio_norm` medio ≈ **2018.49**.  
- `edad_pil_num` media ≈ **33.0** (desvío ~11.4) sobre **solo 4.8%** de filas (no representativo si no se filtra).

**PERSONAS**  
- `anio_norm` medio ≈ **2018.12**.  
- `edad_per_num` media ≈ **31.0** (desvío ~16.26).  
- Banderas: `is_fallecido=1` y `is_lesionado=0` en **100%** de registros → confirma naturaleza de fallecidos.

---

## 5) Observaciones de normalización/categorización
- `tipo_eve_norm`: aparecen valores **`NAN`** en HECHOS (2013), señal de filas no clasificadas; conviene **completar/estandarizar** para no perderlas en agregaciones por tipo.  
- `color_std`, `tipo_veh_norm`: están presentes en VEHICULOS; coherentes con la normalización previa (mapeos numéricos/códigos).  
- `depto_name_std`: disponible en HECHOS, útil para rotular gráficos; en VEHICULOS/PERSONAS se prioriza `depto_code` (80%± cobertura).

---






## **Inciso 2**

In [0]:
# INCISO 2 — Años disponibles por tabla y validación de coincidencia
# Requiere que existan las vistas temporales: `hechos`, `vehiculos`, `personas`
# (quedan creadas por el bloque del inciso 1 sobre `master_unico.csv`)

from pyspark.sql import functions as F

def years_count_df(view_name: str, year_col: str = "anio_norm"):
    """
    Devuelve un DataFrame con: anio, count
    - Filtra solo años válidos (exactamente 4 dígitos).
    - Convierte a entero para ordenar correctamente.
    """
    df = spark.table(view_name)
    if year_col not in df.columns:
        raise ValueError(f"La vista '{view_name}' no tiene la columna '{year_col}'.")
    base = (
        df.select(F.col(year_col).cast("string").alias("y"))
          .filter(F.col("y").rlike(r"^\d{4}$"))
          .withColumn("anio", F.col("y").cast("int"))
    )
    return base.groupBy("anio").count().orderBy("anio")

def table_totals(view_name: str, year_col: str = "anio_norm"):
    """
    Retorna (total_filas, filas_con_anio_valido, df_por_anio)
    """
    df = spark.table(view_name)
    total = df.count()
    df_years = years_count_df(view_name, year_col)
    valid = df_years.agg(F.sum("count").alias("valid")).collect()[0]["valid"] or 0
    return total, valid, df_years

# 1) Conteos por año para cada tabla
tot_h, valid_h, hechos_y = table_totals("hechos")
tot_v, valid_v, vehiculos_y = table_totals("vehiculos")
tot_p, valid_p, personas_y = table_totals("personas")

print("=== Hechos: años disponibles y conteos ===")
display(hechos_y)
print(f"Registros con año válido: {valid_h:,} / total: {tot_h:,}\n")

print("=== Vehículos: años disponibles y conteos ===")
display(vehiculos_y)
print(f"Registros con año válido: {valid_v:,} / total: {tot_v:,}\n")

print("=== Personas (fallecidos): años disponibles y conteos ===")
display(personas_y)
print(f"Registros con año válido: {valid_p:,} / total: {tot_p:,}\n")

# 2) Validación de coincidencia de años entre tablas
H = set(r["anio"] for r in hechos_y.select("anio").distinct().collect())
V = set(r["anio"] for r in vehiculos_y.select("anio").distinct().collect())
P = set(r["anio"] for r in personas_y.select("anio").distinct().collect())

interseccion = sorted(H & V & P)
union_total = sorted(H | V | P)
faltan_en_H = sorted(set(union_total) - H)
faltan_en_V = sorted(set(union_total) - V)
faltan_en_P = sorted(set(union_total) - P)

print("=== Validación de años entre tablas ===")
print(f"Intersección (en TODAS): {interseccion}")
print(f"Unión (en ALGUNA):      {union_total}")
print(f"- HECHOS:    años ausentes vs unión → {faltan_en_H}")
print(f"- VEHICULOS: años ausentes vs unión → {faltan_en_V}")
print(f"- PERSONAS:  años ausentes vs unión → {faltan_en_P}")


=== Hechos: años disponibles y conteos ===


anio,count
2013,6324
2014,5651
2015,6854
2016,7964
2017,5879
2018,6395
2019,7047
2020,6350
2021,8153
2022,7924


Registros con año válido: 76,759 / total: 76,759

=== Vehículos: años disponibles y conteos ===


anio,count
2013,6323
2014,7904
2015,9823
2016,11618
2017,8644
2018,9514
2019,10827
2020,10103
2021,12796
2022,12239


Registros con año válido: 111,988 / total: 111,988

=== Personas (fallecidos): años disponibles y conteos ===


anio,count
2013,9060
2014,8990
2015,10397
2016,11668
2017,8625
2018,9407
2019,10664
2020,8142
2021,10544
2022,10722


Registros con año válido: 109,417 / total: 109,417

=== Validación de años entre tablas ===
Intersección (en TODAS): [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Unión (en ALGUNA):      [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
- HECHOS:    años ausentes vs unión → []
- VEHICULOS: años ausentes vs unión → []
- PERSONAS:  años ausentes vs unión → []


# Inciso 2 — Años disponibles por tabla y validación

## Resumen ejecutivo
- **Cobertura temporal consistente (2013–2023)** en **las tres tablas**: HECHOS, VEHÍCULOS y PERSONAS.
- **Conteos totales**:
  - **HECHOS:** 76,759
  - **VEHÍCULOS:** 111,988
  - **PERSONAS (fallecidos):** 109,417
- **Relaciones útiles** (sobre todo el periodo):
  - ≈ **1.46 vehículos por accidente** (111,988 / 76,759).
  - ≈ **1.43 fallecidos por accidente** (109,417 / 76,759).  
    Esto sugiere múltiples víctimas por hecho en numerosos eventos.

---

## HECHOS — Años y tendencias
- Años presentes y **conteo por año**: 2013–2023, todos con registros.
- **Pico** en **2023** (8,218) y segundo pico en **2021** (8,153).
- **Bajada en 2020** (6,350), coherente con una posible reducción de movilidad durante la pandemia.
- **Patrón general**: leve crecimiento de 2013–2016, oscilación 2017–2019, caída en 2020, repunte marcado 2021–2023.

## VEHÍCULOS — Años y tendencias
- Años presentes y **conteo por año**: 2013–2023, todos con registros.
- **Máximo** en **2021** (12,796), con ligera **corrección** en 2022–2023 (12,239 y 12,197).
- El volumen supera sistemáticamente a HECHOS (≥1 vehículo por evento, con frecuentes >1), lo cual habilita el cálculo de **promedio de vehículos por accidente** por año/departamento.

## PERSONAS (FALLECIDOS) — Años y tendencias
- Años presentes y **conteo por año**: 2013–2023, todos con registros.
- **Mínimo** en **2020** (8,142) y **máximo** en **2023** (11,198).
- El **descriptor** previo indica `is_fallecido=1` e `is_lesionado=0` en la tabla, por lo que esta partición representa **solo fallecidos**. El conteo superior a HECHOS es consistente con **múltiples fallecidos por hecho** en diversos casos.

---

## Validación de coincidencia de años
- **Intersección y unión** de años entre tablas:  
  **[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]**
- **Ausencias por tabla vs unión:**  
  - **HECHOS:** []  
  - **VEHÍCULOS:** []  
  - **PERSONAS:** []
- **Conclusión:** cobertura temporal **perfectamente alineada**; no hay huecos entre tablas, lo cual **facilita joins** por año y comparaciones intertabla para los incisos posteriores.

---

## Implicaciones analíticas
- El alineamiento 2013–2023 permite:
  - Calcular y graficar **accidentes por año y departamento** sin ajustes de ventana temporal.
  - Estimar **vehículos por accidente** y **fallecidos por accidente** por año/departamento con consistencia.
  - Aislar el **efecto 2020** (posible pandemia) y contrastarlo con la **recuperación 2021–2023**.
- Para joins robustos en incisos 8–12, conviene usar llaves normalizadas:  
  `anio_norm`, `mes_norm`, `depto_code` (y opcionalmente `depto_name_std`) y `tipo_eve_norm`.



## **Inicso 3**

In [0]:
# INCISO 3 — Valores distintos de tipo de accidente (usando el master ya cargado)
# Se asume que ya existen las vistas temporales: master_unico, hechos, vehiculos, personas

from pyspark.sql import functions as F

# === 1) Usar HECHOS como fuente canónica de "tipo de accidente" ===
hechos = spark.table("hechos")

# Limpieza defensiva: pasar a string, recortar, upper; tratar "NAN"/"NULL"/vacíos como nulos
hechos_tipo = (
    hechos
    .withColumn("tipo_eve_norm_str", F.upper(F.trim(F.col("tipo_eve_norm").cast("string"))))
    .withColumn(
        "tipo_eve_norm_str",
        F.when(F.col("tipo_eve_norm_str").isin("", "NULL", "NAN", "NONE", "NA"), F.lit(None))
         .otherwise(F.col("tipo_eve_norm_str"))
    )
)

# === 2) Distintos y conteos globales (HECHOS) ===
distinct_tipos = (
    hechos_tipo
    .groupBy("tipo_eve_norm_str")
    .count()
    .orderBy(F.desc("count"))
)

print("=== Distintos 'tipo_eve_norm' con conteo (HECHOS) ===")
distinct_tipos.show(100, truncate=False)   # Ajustar 100 si se desea ver más

# === 3) Solo listado de valores distintos (sin conteos) ===
print("\n=== Valores distintos (solo listado) ===")
(
    hechos_tipo
    .select("tipo_eve_norm_str")
    .distinct()
    .orderBy("tipo_eve_norm_str")
).show(200, truncate=False)

# === 4) (Opcional) Aparición por año para cruzar con inciso 2 ===
por_anio = (
    hechos_tipo
    .groupBy("anio_norm", "tipo_eve_norm_str")
    .count()
    .orderBy("anio_norm", F.desc("count"))
)

print("\n=== Aparición por año (tipo_eve_norm) ===")
por_anio.show(100, truncate=False)

# === 5) Visualizaciones Databricks (usar display para tabla o gráfico) ===
display(distinct_tipos)  # gráfico de barras: eje X = tipo_eve_norm_str, eje Y = count
display(por_anio)        # gráfico de columnas apiladas o tabla por año

# === 6) (Opcional) Validar consistencia entre tablas del master ===
dist_por_tabla = (
    spark.table("master_unico")
    .withColumn("tipo_eve_norm_str", F.upper(F.trim(F.col("tipo_eve_norm").cast("string"))))
    .withColumn(
        "tipo_eve_norm_str",
        F.when(F.col("tipo_eve_norm_str").isin("", "NULL", "NAN", "NONE", "NA"), F.lit(None))
         .otherwise(F.col("tipo_eve_norm_str"))
    )
    .groupBy("source_table", "tipo_eve_norm_str")
    .count()
    .orderBy("source_table", F.desc("count"))
)

# Útil si se quiere revisar si VEHICULOS o PERSONAS traen alguna variante extra
display(dist_por_tabla)


=== Distintos 'tipo_eve_norm' con conteo (HECHOS) ===
+-----------------+-----+
|tipo_eve_norm_str|count|
+-----------------+-----+
|COLISION         |34998|
|ATROPELLO        |21778|
|DERRAPE          |8826 |
|NULL             |6324 |
|CHOQUE           |2808 |
|VUELCO           |1689 |
|EMBARRANCO       |294  |
|CAIDA            |25   |
|IGNORADO         |15   |
|ENCUNET?         |2    |
+-----------------+-----+


=== Valores distintos (solo listado) ===
+-----------------+
|tipo_eve_norm_str|
+-----------------+
|NULL             |
|ATROPELLO        |
|CAIDA            |
|CHOQUE           |
|COLISION         |
|DERRAPE          |
|EMBARRANCO       |
|ENCUNET?         |
|IGNORADO         |
|VUELCO           |
+-----------------+


=== Aparición por año (tipo_eve_norm) ===
+---------+-----------------+-----+
|anio_norm|tipo_eve_norm_str|count|
+---------+-----------------+-----+
|2013     |NULL             |6324 |
|2014     |COLISION         |2345 |
|2014     |ATROPELLO        |2276 |

tipo_eve_norm_str,count
COLISION,34998
ATROPELLO,21778
DERRAPE,8826
null,6324
CHOQUE,2808
VUELCO,1689
EMBARRANCO,294
CAIDA,25
IGNORADO,15
ENCUNET?,2


anio_norm,tipo_eve_norm_str,count
2013,null,6324
2014,COLISION,2345
2014,ATROPELLO,2276
2014,DERRAPE,532
2014,CHOQUE,366
2014,VUELCO,132
2015,COLISION,3006
2015,ATROPELLO,2565
2015,DERRAPE,715
2015,CHOQUE,438


source_table,tipo_eve_norm_str,count
HECHOS,COLISION,34998
HECHOS,ATROPELLO,21778
HECHOS,DERRAPE,8826
HECHOS,null,6324
HECHOS,CHOQUE,2808
HECHOS,VUELCO,1689
HECHOS,EMBARRANCO,294
HECHOS,CAIDA,25
HECHOS,IGNORADO,15
HECHOS,ENCUNET?,2


# Inciso 3 — Tipos de accidente (master_unico)

## Resumen ejecutivo
- En **HECHOS** predominan tres categorías: **COLISION** (45.6%), **ATROPELLO** (28.4%) y **DERRAPE** (11.5%).  
- **2013** aparece con **tipo no clasificado** (NULL) en HECHOS (6,324 filas), lo que explica la cuarta “categoría” por volumen.
- Categorías de baja frecuencia: **CHOQUE**, **VUELCO**, **EMBARRANCO**, **CAIDA**, **ENCUNET?** e **IGNORADO**.
- Se observan **variantes ortográficas/acentos** y **códigos residuales** (p. ej., `ENCUNET?` vs `ENCUNETO`), que conviene normalizar.

---

## Distribución general (HECHOS)
**Top de tipos (con conteos):**
- **COLISION:** 34,998  
- **ATROPELLO:** 21,778  
- **DERRAPE:** 8,826  
- **NULL (no clasificado):** 6,324  
- **CHOQUE:** 2,808  
- **VUELCO:** 1,689  
- **EMBARRANCO:** 294  
- **CAIDA:** 25  
- **IGNORADO:** 15  
- **ENCUNET?**: 2  

> Suma total HECHOS = 76,759 (coincide con el total de filas reportado).

**Valores distintos detectados:**
`{COLISION, ATROPELLO, DERRAPE, CHOQUE, VUELCO, EMBARRANCO, CAIDA, ENCUNET?, IGNORADO, NULL}`

---

## Evolución anual (HECHOS)
- **2013:** todas las filas aparecen como **NULL** en `tipo_eve_norm` (6,324). Indica ausencia de clasificación en la fuente original de ese año.
- **2014–2019:** **COLISION** y **ATROPELLO** se mantienen como los principales tipos; **DERRAPE** permanece tercero.  
- **2020 (pandemia):** **ATROPELLO** baja a 1,608; **COLISION** se mantiene (3,614). Es consistente con menor movilidad peatonal y cambios de patrón de tránsito.
- **2021:** pico reciente de **COLISION** (4,473) y repunte general tras 2020.  
- **2022–2023:** la jerarquía se conserva (**COLISION > ATROPELLO > DERRAPE**) con ligeras variaciones.  
- **Casos residuales por año:**  
  - **IGNORADO:** solo aparece en 2016 (8), 2019 (3) y 2021 (4).  
  - **EMBARRANCO/CAIDA/ENCUNET?**: muy esporádicos, con recuentos bajos (p. ej., **ENCUNET?** solo 2 en 2022).

---

## Consistencia entre tablas del master
- En **PERSONAS** y **VEHICULOS** también domina **COLISION**, seguido de **ATROPELLO** y **DERRAPE**.  
- Los **volúmenes mayores** en PERSONAS/VEHICULOS respecto a HECHOS son esperables (múltiples personas/vehículos por accidente).  
- Se detecta **inconsistencia de rótulos**:
  - HECHOS: `ENCUNET?`  
  - VEHICULOS: `ENCUNETO`  
  → Ambas deben mapearse a un **único valor estándar** (sugerencia: **ENCUNETADO** o **ENCUNETAMIENTO**).

---

## Calidad de datos y normalización recomendada
1. **Acentos/ortografía**  
   - Unificar **COLISION ↔ COLISIÓN**, **EMBARRANCO ↔ EMBARRANCÓ**, **CAIDA ↔ CAÍDA**, etc., en **mayúsculas sin acento** para estabilidad (o, si se prefiere, con acento pero de forma **consistente**).
2. **Valores residuales**  
   - Mapear `ENCUNET?`/`ENCUNETO` → **ENCUNETADO** (o el estándar acordado).  
   - Mantener **IGNORADO** como categoría explícita o reclasificarla en **OTRO/IGNORADO** si se desea limpiar análisis.
3. **Valores nulos**  
   - **2013:** `tipo_eve_norm` es **NULL** en HECHOS. Para análisis por tipo, **excluir** o **separar** 2013 como “no clasificado”.  
   - Documentar este sesgo cuando se comparen proporciones por tipo a través de años.

---

## Implicaciones analíticas
- Para métricas comparables en el tiempo (incisos 5, 8, 9, 12, etc.), conviene:
  - **Filtrar `tipo_eve_norm` no nulo** o **imputar** 2013 como “DESCONOCIDO/NO CLASIFICADO” (no mezclar con tipos reales).  
  - **Aplicar el diccionario de normalización** antes de agrupar (asegura que no se “divida” una misma categoría por diferencias de escritura).
- La tríada **COLISION–ATROPELLO–DERRAPE** concentra >85% de HECHOS (sin 2013 nulos), por lo que explica la mayor parte de las tendencias.




## **Inciso 4**

In [0]:
# INCISO 4 — ¿Cuántos departamentos únicos aparecen en las bases?
# Usa la tabla unificada "master_unico" ya cargada en la sesión.

from pyspark.sql import functions as F

# 1) DataFrame base
df = spark.table("master_unico")

# Normalización suave del nombre (por si hay espacios/acentos inconsistentes)
df_norm = (
    df.withColumn("depto_name_std", F.upper(F.trim(F.col("depto_name_std"))))
)

# 2) Conteo por 'source_table' (HECHOS, PERSONAS, VEHICULOS)
by_source = (
    df_norm.groupBy("source_table")
           .agg(
               F.countDistinct("depto_code").alias("distinct_depto_code"),
               F.countDistinct("depto_name_std").alias("distinct_depto_name_std")
           )
           .orderBy("source_table")
)

print("=== Departamentos únicos por base (códigos y nombres) ===")
display(by_source)

# 3) Conteo TOTAL en el master_unico (sin separar por base)
total_dist = df_norm.agg(
    F.countDistinct("depto_code").alias("distinct_depto_code_total"),
    F.countDistinct("depto_name_std").alias("distinct_depto_name_std_total")
)

print("\n=== Departamentos únicos en TODAS las bases (total master_unico) ===")
total_dist.show(truncate=False)

# 4) (Opcional) Listado detallado para revisión de cobertura
print("\n=== Listado de CÓDIGOS de departamento (con frecuencia) ===")
codes_counts = (
    df_norm.filter(F.col("depto_code").isNotNull())
           .groupBy("depto_code")
           .agg(F.count("*").alias("count"))
           .orderBy(F.col("depto_code").cast("int"))
)
display(codes_counts)

print("\n=== Listado de NOMBRES normalizados (con frecuencia) ===")
names_counts = (
    df_norm.filter(F.col("depto_name_std").isNotNull())
           .groupBy("depto_name_std")
           .agg(F.count("*").alias("count"))
           .orderBy("depto_name_std")
)
display(names_counts)


=== Departamentos únicos por base (códigos y nombres) ===


source_table,distinct_depto_code,distinct_depto_name_std
HECHOS,22,22
PERSONAS,22,22
VEHICULOS,22,22



=== Departamentos únicos en TODAS las bases (total master_unico) ===
+-------------------------+-----------------------------+
|distinct_depto_code_total|distinct_depto_name_std_total|
+-------------------------+-----------------------------+
|22                       |22                           |
+-------------------------+-----------------------------+


=== Listado de CÓDIGOS de departamento (con frecuencia) ===


depto_code,count
1,85475
2,5679
3,7621
4,9361
5,18038
6,8202
7,4535
8,2307
9,8360
10,8231



=== Listado de NOMBRES normalizados (con frecuencia) ===


depto_name_std,count
ALTA VERAPAZ,10718
BAJA VERAPAZ,4388
CHIMALTENANGO,9361
CHIQUIMULA,5564
EL PROGRESO,5679
ESCUINTLA,18038
GUATEMALA,85475
HUEHUETENANGO,5300
IZABAL,8530
JALAPA,3561


## Inciso 4 — Departamentos únicos y distribución (master_unico)

### Cobertura por tabla
Los datos muestran **cobertura completa** de departamentos en las tres fuentes:
- **HECHOS:** 22 códigos únicos / 22 nombres únicos  
- **PERSONAS:** 22 / 22  
- **VEHÍCULOS:** 22 / 22  

En el total del `master_unico` también se mantiene la consistencia:
- **Total:** 22 códigos únicos / 22 nombres únicos

> Interpretación: las tres capas comparten el mismo universo de 22 departamentos; no hay departamentos “fantasma” exclusivos de alguna tabla ni faltantes en otra.

---

### Distribución por departamento (acumulado de todas las tablas)
**Top y bottom por volumen (cuentas sobre registros con `depto_code` no nulo):**

- **Más alto:**  
  - **GUATEMALA:** 85,475 (≈36% del total con código)  
  - **ESCUINTLA:** 18,038  
  - **ALTA VERAPAZ:** 10,718  
  - **IZABAL:** 8,530  
  - **PETEN:** 8,110  
  - **QUETZALTENANGO:** 8,360  
  - **SUCHITEPEQUEZ:** 8,231  

- **Más bajo:**  
  - **TOTONICAPAN:** 2,307  
  - **JALAPA:** 3,561  
  - **SOLOLA:** 4,535  
  - **BAJA VERAPAZ:** 4,388  

> Observación principal: existe un **sesgo metropolitano** evidente; el departamento de Guatemala concentra ~**36%** de todos los registros que traen código de departamento. Esto es consistente con mayor densidad poblacional, parque vehicular y flujo vial.

---

### Consistencia código–nombre
- Los **nombres están normalizados** (mayúsculas y sin acentos): `PETEN`, `QUICHE`, `SACATEPEQUEZ`, etc.  
- **No aparecen variantes duplicadas** (p. ej., `PETÉN` vs `PETEN`), lo que **evita doble conteo** en agregaciones por nombre.
- El conteo agregado por `depto_code` (235,666) coincide con la suma de filas con código presente en cada subtabla:
  - HECHOS (`depto_code` no nulo): 60,617  
  - VEHÍCULOS: 87,552  
  - PERSONAS: 87,497  
  - **Total:** 235,666

> Implicación: se puede agrupar indistintamente por `depto_code` o por `depto_name_std` sin efectos de duplicidad por etiquetas.

---

### Recomendaciones para los incisos siguientes
- Usar **`depto_code`** como clave primaria territorial en joins y agregaciones, y **mostrar `depto_name_std`** solo como etiqueta.
- Mantener los filtros de **no nulos** en `depto_code` para que las tasas y comparativos entre departamentos sean consistentes.
- Para visualizaciones, ordenar por conteo descendente para resaltar la asimetría (Guatemala vs resto).

---
**Tabla de apoyo (cuentas por `depto_code`):**
- 1: 85,475  
- 2: 5,679  
- 3: 7,621  
- 4: 9,361  
- 5: 18,038  
- 6: 8,202  
- 7: 4,535  
- 8: 2,307  
- 9: 8,360  
- 10: 8,231  
- 11: 6,485  
- 12: 5,974  
- 13: 5,300  
- 14: 5,691  
- 15: 4,388  
- 16: 10,718  
- 17: 8,110  
- 18: 8,530  
- 19: 6,854  
- 20: 5,564  
- 21: 3,561  
- 22: 6,682

**Tabla de apoyo (cuentas por `depto_name_std`):**
- GUATEMALA: 85,475  
- ESCUINTLA: 18,038  
- ALTA VERAPAZ: 10,718  
- IZABAL: 8,530  
- PETEN: 8,110  
- QUETZALTENANGO: 8,360  
- SUCHITEPEQUEZ: 8,231  
- SAN MARCOS: 5,974  
- RETALHULEU: 6,485  
- QUICHE: 5,691  
- HUEHUETENANGO: 5,300  
- CHIMALTENANGO: 9,361  
- SANTA ROSA: 8,202  
- JUTIAPA: 6,682  
- ZACAPA: 6,854  
- CHIQUIMULA: 5,564  
- SACATEPEQUEZ: 7,621  
- BAJA VERAPAZ: 4,388  
- SOLOLA: 4,535  
- JALAPA: 3,561  
- SUCHITEPEQUEZ: 8,231  
- TOTONICAPAN: 2,307



## **Inciso 5**

In [0]:
# INCISO 5 — Total de ACCIDENTES por AÑO y DEPARTAMENTO (usando la tabla HECHOS)

from pyspark.sql import functions as F

# 1) Agregar: accidentes por año y departamento
accidentes_anio_depto = (
    spark.table("hechos")  # vista creada en el inciso 1
    .select("anio_norm", "depto_code", "depto_name_std")
    .where(F.col("anio_norm").isNotNull() & F.col("depto_code").isNotNull())
    .groupBy("anio_norm", "depto_code", "depto_name_std")
    .agg(F.count(F.lit(1)).alias("accidentes"))
)

# 2) Mostrar resultado y usar 'display' como gráfico de barras
#    En la UI: cambiar a "Plot" -> "Bar chart"
#      - Keys (x): anio_norm
#      - Values (y): accidentes
#      - Series group / Group by: depto_name_std (o depto_code)
display(accidentes_anio_depto.orderBy("anio_norm", "depto_code"))

# (Opcional) Versión más legible: solo TOP 10 departamentos por volumen total
top10_deptos = (
    accidentes_anio_depto.groupBy("depto_code", "depto_name_std")
    .agg(F.sum("accidentes").alias("total_depto"))
    .orderBy(F.desc("total_depto"))
    .limit(10)
)

accidentes_top10 = accidentes_anio_depto.join(F.broadcast(top10_deptos.select("depto_code")), on="depto_code")
display(accidentes_top10.orderBy("anio_norm", F.desc("accidentes")))

# (Sanity check) El total por año debe coincidir con el conteo de HECHOS por año:
totales_por_anio = accidentes_anio_depto.groupBy("anio_norm").agg(F.sum("accidentes").alias("accidentes"))
display(totales_por_anio.orderBy("anio_norm"))


anio_norm,depto_code,depto_name_std,accidentes
2013,1,GUATEMALA,2267
2013,10,SUCHITEPEQUEZ,269
2013,11,RETALHULEU,133
2013,12,SAN MARCOS,221
2013,13,HUEHUETENANGO,201
2013,14,QUICHE,151
2013,15,BAJA VERAPAZ,100
2013,16,ALTA VERAPAZ,219
2013,17,PETEN,214
2013,18,IZABAL,234


depto_code,anio_norm,depto_name_std,accidentes
1,2013,GUATEMALA,2267
5,2013,ESCUINTLA,424
9,2013,QUETZALTENANGO,288
10,2013,SUCHITEPEQUEZ,269
18,2013,IZABAL,234
16,2013,ALTA VERAPAZ,219
17,2013,PETEN,214
6,2013,SANTA ROSA,214
4,2013,CHIMALTENANGO,212
3,2013,SACATEPEQUEZ,142


anio_norm,accidentes
2013,6324
2014,5651
2015,6854
2016,7964
2017,5879
2018,6395
2019,7047
2020,6350
2021,8153


# Inciso 5 — Análisis de accidentes por **año** y **departamento**

## Resumen de volumen anual (HECHOS)
Los conteos anuales reportados muestran una evolución con caída en 2020 y pico en 2021:

- **2013:** 6,324  
- **2014:** 5,651  
- **2015:** 6,854  
- **2016:** 7,964  
- **2017:** 5,879  
- **2018:** 6,395  
- **2019:** 7,047  
- **2020:** 6,350  
- **2021:** 8,153  ← mayor valor del periodo

**Lectura rápida:** el descenso de 2020 es consistente con restricciones de movilidad; 2021 rebota a un máximo.

---

## Patrones por departamento (2013–2021)

### Concentración metropolitana
- **Guatemala (Depto 1)** domina todos los años:  
  2013: 2,267 → 2014: 1,912 → 2015: 2,273 → 2016: 2,619 → 2017: 2,128 →  
  2018: 2,722 → 2019: 3,017 → 2020: 2,676 → **2021: 3,450**.  
  **Interpretación:** mayor densidad vehicular y flujo urbano sostienen el liderazgo y el rebote post-2020.

### Segundo bloque de alta incidencia
- **Escuintla (5)** se mantiene alto: 424 → 444 → 569 → **627** → 461 → 407 → 452 → 440 → **615**.  
  Presenta un **pico en 2016** y un **nuevo máximo en 2021**.
- **Quetzaltenango (9)** y **Suchitepéquez (10)** conservan volúmenes medios-altos y estables (p. ej., Quetzaltenango: 288, 255, 316, 323, 172, 155, 191, 197, 253).

### Departamentos con tendencia al alza moderada
- **Alta Verapaz (16)**: 219 → 223 → 315 → 353 → 247 → 265 → 317 → 296 → **380**.  
  **Lectura:** incremento estructural tras 2015–2016 y nuevo máximo en 2021.

### Cola baja y estabilidad
- **Totonicapán (8)** es el de menor incidencia en casi todos los años:  
  81 → 82 → **69** → 76 → **44** → 47 → 63 → 49 → 77.  
- **Jalapa (21)** y **Sololá (7)** también se ubican sistemáticamente en la parte baja del ranking.

---

## Picos por año (muestras)
- **2013:** Guatemala (2,267) muy por encima; Escuintla (424) y Quetzaltenango (288) siguen.  
- **2016:** máximos anuales en varios departamentos (p. ej., Escuintla 627, Chimaltenango 420).  
- **2021:** máximos recientes generalizados (Guatemala 3,450; Escuintla 615; Alta Verapaz 380).

---

## Señales temporales relevantes
1. **Efecto 2020:** caída transversal (p. ej., Guatemala 3,017 → 2,676) coherente con menor movilidad.  
2. **Rebote 2021:** incremento notable en casi todos los departamentos, alcanzando máximos del periodo.  
3. **Persistencia espacial:** la jerarquía territorial cambia poco; Guatemala y Escuintla sostienen el podio.

---

## Posibles factores explicativos
- **Exposición al riesgo:** densidad poblacional, parque vehicular y ejes viales nacionales (Guatemala, Escuintla).  
- **Actividad económica y logística:** corredores de carga y puertos incrementan flujo (Escuintla, Izabal).  
- **Orografía y clima:** tramos con pendientes/curvas o lluvias pueden elevar siniestralidad (Alta Verapaz).  
- **Políticas y controles:** variaciones locales en fiscalización o infraestructura inciden en tendencias.

---

## Recomendaciones de visualización (en Databricks `display`)
- **Barras apiladas:** `anio_norm` en el eje X, `accidentes` en Y y **serie** por `depto_name_std` para comparar composición anual.  
- **Barras agrupadas:** filtrar a los **Top 10** departamentos por volumen total para claridad.  
- **Línea anual:** sumar por año y graficar tendencia global (confirma el valle 2020 y el pico 2021).

---

## Notas de calidad
- Los totales por año reflejados concuerdan con la suma por departamentos del mismo año en las salidas mostradas.  
- La cobertura del análisis se centra en 2013–2021 según el extracto entregado; si existen 2022–2023, conviene incluirlos en la misma estructura para completar la serie.


## **Inciso 6**

In [0]:
# INCISO 6 (corregido) — Día de la semana con más accidentes.
# Recalcula el día de la semana desde (anio_norm, mes_norm, dia_norm).
from pyspark.sql import functions as F

hechos = spark.table("hechos")

# 1) Resolver año objetivo (2024 si existe; si no, último disponible)
years = (
    hechos
    .select(F.col("anio_norm").cast("int").alias("anio"))
    .where(F.col("anio").isNotNull())
    .distinct()
    .orderBy("anio")
    .collect()
)
years_list = [r["anio"] for r in years]
if not years_list:
    raise ValueError("No hay años disponibles en 'hechos'.")
target_year = 2024 if 2024 in years_list else max(years_list)

# 2) Construir fecha válida y filtrar al año objetivo
base = (
    hechos
    .withColumn("anio_i", F.col("anio_norm").cast("int"))
    .withColumn("mes_i",  F.col("mes_norm").cast("int"))
    .withColumn("dia_i",  F.col("dia_norm").cast("int"))
    .withColumn("fecha",  F.to_date(F.concat_ws("-", "anio_i", "mes_i", "dia_i"), "yyyy-M-d"))
    .where(F.col("fecha").isNotNull())
    .where(F.year("fecha") == F.lit(target_year))
)

# 3) dayofweek(fecha): Spark da 1=Domingo..7=Sábado.
#    Convertimos a lunes=1..domingo=7 para mantener el orden habitual.
dow_sun1 = F.dayofweek("fecha")
dow_mon1 = ((dow_sun1 + F.lit(5)) % F.lit(7)) + F.lit(1)

dist_dow = (
    base
    .withColumn("dow_num", dow_mon1.cast("int"))
    .withColumn(
        "dow_label",
        F.when(F.col("dow_num")==1,"LUNES")
         .when(F.col("dow_num")==2,"MARTES")
         .when(F.col("dow_num")==3,"MIÉRCOLES")
         .when(F.col("dow_num")==4,"JUEVES")
         .when(F.col("dow_num")==5,"VIERNES")
         .when(F.col("dow_num")==6,"SÁBADO")
         .when(F.col("dow_num")==7,"DOMINGO")
    )
    .groupBy("dow_num", "dow_label")
    .agg(F.count(F.lit(1)).alias("accidentes"))
    .orderBy("dow_num")
)

print(f"[INFO] Año analizado: {target_year}")
display(dist_dow.select("dow_num", "dow_label", "accidentes"))  # Configura como 'Bar chart': X=dow_label, Y=accidentes

# 4) Día con más accidentes
top = dist_dow.orderBy(F.desc("accidentes")).limit(1).collect()
if top:
    print(f"Día con más accidentes en {target_year}: {top[0]['dow_label']} ({top[0]['accidentes']})")
else:
    print(f"No hay datos para {target_year} tras construir la fecha.")


[INFO] Año analizado: 2023


dow_num,dow_label,accidentes
1,LUNES,1053
2,MARTES,932
3,MIÉRCOLES,924
4,JUEVES,959
5,VIERNES,1173
6,SÁBADO,1557
7,DOMINGO,1620


Día con más accidentes en 2023: DOMINGO (1620)


# Inciso 6 — Accidentes por día de la semana (análisis)

## Resumen
- **Total analizado:** 8,218 accidentes.
- **Día con más accidentes:** **DOMINGO (1,620; 19.7%)**.
- **Segundo lugar:** **SÁBADO (1,557; 18.9%)**.
- **Tercer lugar:** **VIERNES (1,173; 14.3%)**.
- **Menor registro:** **MIÉRCOLES (924; 11.2%)**.

## Tabla de distribución
| Día         | Accidentes | % del total |
|-------------|------------:|------------:|
| LUNES       | 1,053       | 12.8%       |
| MARTES      |   932       | 11.3%       |
| MIÉRCOLES   |   924       | 11.2%       |
| JUEVES      |   959       | 11.7%       |
| VIERNES     | 1,173       | 14.3%       |
| SÁBADO      | 1,557       | 18.9%       |
| **DOMINGO** | **1,620**   | **19.7%**   |

## Patrones clave
- **Finde (Sábado+Domingo) = 3,177 accidentes (38.7%)** del total.
- **Viernes+Finde = 4,350 (52.9%)**, es decir, **más de la mitad** ocurre entre viernes y domingo.
- El **DOMINGO** supera en **≈67.5%** el promedio diario de lunes a jueves.
- La brecha **DOMINGO vs. día mínimo (MIÉRCOLES)** es de **+696 accidentes**.

## Posibles razones del pico en fin de semana
1. **Movilidad recreativa y social:** mayor volumen de viajes no laborales (salidas familiares, ocio, turismo de cercanía).
2. **Fatiga y conducción nocturna:** retornos del fin de semana, trayectos largos y conducción en la noche/tarde-noche.
3. **Consumo de alcohol y distracciones:** mayor exposición a factores de riesgo en actividades sociales.
4. **Cambios en la mezcla vehicular:** más motocicletas y autos particulares en ocio, distintos patrones a los días laborales.
5. **Flujos de retorno los domingos:** concentración de viajes de regreso hacia áreas urbanas genera **picos vespertinos/nocturnos**.

## Recomendaciones operativas
- **Refuerzo de controles** (alcoholímetro, velocidad) y **señalización** en **viernes noche a domingo noche**.
- **Campañas preventivas** enfocadas en **retorno dominical** (descanso, pausas, conductor designado).
- **Gestión de tráfico y patrullaje** focalizado en **corredores de salida/retorno** y zonas de ocio.




In [0]:
# INCISO 7 — Distribución de accidentes por hora del día en el Municipio de Guatemala
# Dataset: master_unico (con columnas normalizadas)
# Criterio de filtro: Municipio de Guatemala → depto_code = 1 y muni_code = 101

from pyspark.sql import functions as F

# 1) Filtrar hechos del Municipio de Guatemala y sanear hora
hechos = spark.table("master_unico").where(F.col("source_table") == "HECHOS")

hechos_gua = (
    hechos
    .where( (F.col("depto_code") == 1) & (F.col("muni_code") == 101) )
    .withColumn("hora_clean_int", F.col("hora_clean").cast("int"))
    .filter(F.col("hora_clean_int").between(0, 23))
)

print(f"[INFO] Municipio de Guatemala — filas consideradas: {hechos_gua.count():,}")

# 2) Distribución por hora (conteo)
dist_hora = (
    hechos_gua
    .groupBy("hora_clean_int")
    .agg(F.count(F.lit(1)).alias("accidentes"))
    .orderBy("hora_clean_int")
)

# 3) Mostrar tabla y (en Databricks) graficar
# En la tabla resultante, haga clic en 'Plot' → 'Bar chart'
#    - Keys (X): hora_clean_int
#    - Values (Y): accidentes
#    - Aggregation: SUM (por defecto)
display(dist_hora)

# 4) Opción alternativa: histograma real de una sola columna (Databricks)
#    - Haga clic en 'Plot' → 'Histogram'
#    - Values: hora_clean
display(
    hechos_gua.select(F.col("hora_clean_int").alias("hora_clean"))
)

# 5) (Opcional) Top 3 horas con más accidentes (impresión rápida)
top3 = dist_hora.orderBy(F.desc("accidentes")).limit(3).collect()
if top3:
    print("Top 3 horas (Municipio de Guatemala):")
    for r in top3:
        print(f"  - {int(r['hora_clean_int']):02d}:00 → {r['accidentes']} accidentes")


[INFO] Municipio de Guatemala — filas consideradas: 13,003


hora_clean_int,accidentes
0,560
1,569
2,452
3,377
4,284
5,330
6,358
7,451
8,408
9,453


hora_clean
5
15
7
22
11
17
7
9
0
5


Top 3 horas (Municipio de Guatemala):
  - 21:00 → 828 accidentes
  - 23:00 → 798 accidentes
  - 22:00 → 774 accidentes


Inciso 7 — Distribución por hora del día (Municipio de Guatemala)

Total de accidentes considerados: 13,003

Picos horarios

21:00 → 828 (≈ 6.4%)

23:00 → 798 (≈ 6.1%)

22:00 → 774 (≈ 6.0%)

20:00 → 768 (≈ 5.9%)

19:00 → 764 (≈ 5.9%)

La franja 19:00–22:00 concentra 3,134 accidentes (≈ 24.1% del total).

Horas con menor incidencia

04:00 → 284 (mínimo diario)

Franjas horarias (agregadas)

Madrugada [0–5]: 2,572 (≈ 19.8%)

Mañana [6–11]: 2,651 (≈ 20.4%)

Tarde [12–17]: 3,174 (≈ 24.4%)

Noche [18–23]: 4,606 (≈ 35.4%)

Lecturas clave

La noche (18–23) es la franja con mayor siniestralidad (más de un tercio de los casos), con un máximo entre 19:00 y 22:00.

La madrugada (0–5) concentra ~1 de cada 5 eventos, con el mínimo a las 04:00.

La incidencia aumenta desde las 17:00 y se mantiene elevada hasta cerca de medianoche.

